In [1]:
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

from sklearn.model_selection import train_test_split
from pylab import *

import pickle
import loadnotebook
from predictionhelper import *

from bayes_opt import BayesianOptimization
import sklearn.metrics as metric

import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from predictionhelper.ipynb


In [ ]:
sets = [x for x in range(1, 34)]
demo_config = {6 : sets}

df_data = get_data(config=demo_config, pure=True, refresh=False).reset_index(drop=True)
print(len(df_data))

In [ ]:
pci_data = df_data[df_data["PCI"].isin(whitelist_PCI)]
beam_columns = [c for c in df_data if "beam" in c]
pci_data = pci_data.drop(["RSRP", "RSRQ", "SNR"]+beam_columns, axis=1)
pci_data = pci_data.drop_duplicates()
pci_data = pci_data.dropna()

In [4]:
pci_train, pci_test = pd.DataFrame(), pd.DataFrame()
pci_train_dict, pci_test_dict = {}, {}
for p in demo_config :
    for s in demo_config[p] :
        a, b = train_test_split(pci_data[pci_data.set==s], test_size=0.3, random_state=32)
        pci_train = pci_train.append(a)
        pci_test = pci_test.append(b)  
        pci_train_dict[(p, s)] = a
        pci_test_dict[(p, s)] = b
print(len(pci_train), len(pci_test))

6356 2745


In [5]:
x_pci_train = pci_train.drop(["PCI"], axis=1)
y_pci_train = np.array(pci_train.PCI.apply(lambda x : pci_encode[x]).values.tolist())
x_pci_test = pci_test.drop(["PCI"], axis=1)
y_pci_test = np.array(pci_test.PCI.apply(lambda x : pci_encode[x]).values.tolist())

x_pci_train_dict, y_pci_train_dict, x_pci_test_dict, y_pci_test_dict = {}, {}, {}, {}
for p in demo_config :
    for s in demo_config[p] :
        a, b = pci_train_dict[(p,s)], pci_test_dict[(p,s)]
        x_pci_train_dict[(p, s)] = a.drop(["PCI"], axis=1)
        y_pci_train_dict[(p, s)] = np.array(a.PCI.apply(lambda x : pci_encode[x]).values.tolist())
        x_pci_test_dict[(p, s)] = b.drop(["PCI"], axis=1)
        y_pci_test_dict[(p, s)] = np.array(b.PCI.apply(lambda x : pci_encode[x]).values.tolist())

# SVM 

In [6]:
from sklearn import svm

class svm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {}
        params['C'] = param['C'] if param['C'] > 0 else 0.1
        params['random_state'] = int(param['random_state'])
        return params
        
    def evaluate(self, C=1.0, random_state=0):
        params = {'C':C, 'random_state':random_state}
        params = self.clean_param(params)

        model = svm.LinearSVC(**params)
        model.fit(self.x_train, self.y_train)
        y_pred = model.predict(self.x_test)
        predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(self.y_test, predictions)
        return accuracy

In [7]:
svm_params = {'random_state':0}
model = svm.LinearSVC(**svm_params)
model.fit(x_pci_train, y_pci_train)

y_pci_pred = model.predict(x_pci_test)
predictions = [round(value) for value in y_pci_pred]
accuracy = accuracy_score(y_pci_test, predictions)
print(1-accuracy)

0.26775956284153


# KNN 

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
class knn_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.weights = ['uniform', 'distance']
        self.algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
        
    def clean_param(self, param) :
        params = {'n_neighbors':7}
        params['weights'] = self.weights[int(param['weight'])]
        params['algorithm'] = self.algorithms[int(param['algorithm'])]
        params['leaf_size'] = int(param['leaf_size'])
        params['p'] = int(param['p'])
        return params
        
    def evaluate(self, weight, algorithm, leaf_size=100, p=2):
        params = {}
        params['weight'] = weight
        params['algorithm'] = algorithm
        params['leaf_size'] = int(leaf_size)
        params['p'] = int(p)
        params = self.clean_param(params)

        model = KNeighborsClassifier(**params)
        model.fit(self.x_train, self.y_train)
        y_pci_pred = model.predict(self.x_test)
        predictions = [round(value) for value in y_pci_pred]
        accuracy = accuracy_score(self.y_test, predictions)
        return accuracy

In [69]:
# kt = knn_target(x_pci_train[location_col], y_pci_train, 
#                 x_pci_test[location_col], y_pci_test)
# kBO = BayesianOptimization(kt.evaluate, {'weight': (0, 1),
#                                         'algorithm' : (0, 3),
#                                         'leaf_size' : (5, 50),
#                                         'p': (1, 2),},
#                             random_state = 1)

# kBO.maximize(init_points=20, n_iter=1)

In [13]:
# params = kt.clean_param(kBO.res['max']['max_params'])
# params

{'n_neighbors': 7,
 'weights': 'uniform',
 'algorithm': 'kd_tree',
 'leaf_size': 49,
 'p': 1}

In [10]:
knn_params = {'n_neighbors': 6,
 'weights': 'uniform',
 'algorithm': 'kd_tree',
 'leaf_size': 49,
 'p': 1}

In [11]:
model = KNeighborsClassifier(**knn_params)
model.fit(x_pci_train, y_pci_train)

y_pci_pred = model.predict(x_pci_test)
predictions = [round(value) for value in y_pci_pred]
accuracy = accuracy_score(y_pci_test, predictions)
print(1-accuracy)

0.3067395264116576


# XGBoost 

In [12]:
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [13]:
class xgboost_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        booster_dict = {1:'gbtree', 2:'gblinear', 3:'dart'}
        params = {'base_score':0.5, 'booster':'gbtree', 'missing':None, 'n_estimators':100, 
                  'n_jobs':1, 'objective':'multi:softmax', 'random_state':1, 
                  'reg_lambda':1, 'alpha':0, 'scale_pos_weight':1, 
                  'subsample':1, 'colsample_bytree':1, 'colsample_bylevel':1}

        params['learning_rate'] = param['learning_rate']/100
        params['booster'] = booster_dict[int(param['booster'])]
        params['gamma'] = param['gamma']
        params['max_depth'] = int(param['max_depth'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_delta_weight'] = int(param['max_delta_weight'])
        params['rate_drop'] = param['rate_drop']
        return params
        
    def evaluate(self, learning_rate, booster, gamma, max_depth,  
                     min_child_weight, max_delta_weight, rate_drop):

        params = {}
        params['learning_rate'] = learning_rate
        params['booster'] = booster
        params['gamma'] = gamma
        params['max_depth'] = max_depth
        params['min_child_weight'] = min_child_weight
        params['max_delta_weight'] = max_delta_weight
        params['rate_drop'] = rate_drop
        params = self.clean_param(params)

        xgb_model = XGBClassifier(**params)
        xgb_model.fit(self.x_train, self.y_train)
        y_pci_pred = xgb_model.predict(self.x_test)
        predictions = [round(value) for value in y_pci_pred]
        accuracy = accuracy_score(self.y_test, predictions)
        return accuracy

In [30]:
# xt = xgboost_target(x_pci_train, y_pci_train, x_pci_test, y_pci_test)
# xgbBO = BayesianOptimization(xt.evaluate, {'learning_rate': (1, 12),
#                                             'booster' : (1, 3),
#                                             'gamma' : (0, 50),
#                                             'max_depth': (3, 12),
#                                             'min_child_weight': (1, 1),
#                                             'max_delta_weight': (1, 20),
#                                             'rate_drop': (0, 1)},
#                             random_state = 1)

# xgbBO.maximize(init_points=10, n_iter=5)

Initialization
---------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   booster |     gamma |   learning_rate |   max_delta_weight |   max_depth |   min_child_weight |   rate_drop | 
    1 | 00m03s |    0.73807 |    1.8384 |   40.0372 |          5.5872 |             1.3680 |      3.8851 |             1.0000 |      0.1023 | 
    2 | 00m00s |    0.69945 |    2.3704 |   48.4131 |          8.9236 |            13.8979 |      6.7900 |             1.0000 |      0.4141 | 
    3 | 00m09s |    0.74026 |    1.4089 |   15.6712 |          1.0013 |             5.0209 |     11.6210 |             1.0000 |      0.6944 | 
    4 | 00m00s |    0.66485 |    2.7562 |   34.6161 |          4.3257 |             6.0454 |      7.7985 |             1.0000 |      0.4142 | 
    5 | 00m09s |    0.73151 |    1.0548 |   43.8195 |          2.6143 |            10.3399 |      9.2269 |             1.0000 | 

In [31]:
# params = xt.clean_param(xgbBO.res['max']['max_params'])
# xgb_model = XGBClassifier(**params)
# xgb_model.fit(x_pci_train, y_pci_train)

# y_pci_pred = xgb_model.predict(x_pci_test)
# predictions = [round(value) for value in y_pci_pred]
# accuracy = accuracy_score(y_pci_test, predictions)
# print(1-accuracy)

0.24553734061930788


In [14]:
xgboost_params = {'base_score': 0.5,
 'booster': 'gbtree',
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'objective': 'multi:softmax',
 'random_state': 1,
 'reg_lambda': 1,
 'alpha': 0,
 'scale_pos_weight': 1,
 'subsample': 1,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'learning_rate': 0.0536444221653737,
 'gamma': 8.491520978228445,
 'max_depth': 3,
 'min_child_weight': 1,
 'max_delta_weight': 12,
 'rate_drop': 0.9445947559908133}

In [15]:
xgb_model = XGBClassifier(**xgboost_params)
xgb_model.fit(x_pci_train, y_pci_train)

y_pci_pred = xgb_model.predict(x_pci_test)
predictions = [round(value) for value in y_pci_pred]
accuracy = accuracy_score(y_pci_test, predictions)
print(1-accuracy)

0.24553734061930788


# LGBM 

In [16]:
import lightgbm
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [17]:
class lgbm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'boosting_type':'gbdt', 'class_weight':None, 'colsample_bytree':1.0, 
                  'importance_type':'split', 
                  'min_child_samples':20, 'min_split_gain':0.0, 'n_estimators':100, 'objective':None,
                  'random_state':0, 'reg_alpha':0.0, 'reg_lambda':0.0, 'silent':True,
                  'subsample':1.0, 'subsample_for_bin':200000, 'subsample_freq':0}
        params['num_leaves'] = int(param['num_leaves'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_depth'] = int(param['max_depth'])
        params['learning_rate'] = param['learning_rate'] / 100
        return params
        
    def evaluate(self, min_child_weight, learning_rate, max_depth, num_leaves):
        params = {'num_leaves':num_leaves, 
                  'min_child_weight':min_child_weight, 
                  'max_depth':max_depth, 
                  'learning_rate':learning_rate}
        
        params = self.clean_param(params)

        lgbm_model = LGBMClassifier(**params )
        lgbm_model.fit(self.x_train, self.y_train)
        y_pci_pred = lgbm_model.predict(self.x_test)
        predictions = [round(value) for value in y_pci_pred]
        accuracy = accuracy_score(self.y_test, predictions)
        return accuracy

In [43]:
# lt = lgbm_target(x_pci_train, y_pci_train, x_pci_test, y_pci_test)
# lgbmBO = BayesianOptimization(lt.evaluate, {'min_child_weight': (0.01, 1),
#                                               'learning_rate': (1, 10),
#                                               'max_depth': (-1, 15),
#                                               'num_leaves': (5, 50)}, 
#                              random_state=3)

# lgbmBO.maximize(init_points=20, n_iter=5)

Initialization
-------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   learning_rate |   max_depth |   min_child_weight |   num_leaves | 
    1 | 00m01s |    0.74681 |          3.5517 |      5.5043 |             0.5508 |      21.8120 | 
    2 | 00m01s |    0.72750 |          7.2382 |      6.5030 |             0.7081 |      35.0661 | 
    3 | 00m00s |    0.75519 |          4.9641 |      3.3078 |             0.2909 |      20.2932 | 
    4 | 00m00s |    0.74608 |          2.4118 |      3.6687 |             0.5108 |      30.7757 | 
    5 | 00m01s |    0.74281 |          5.9018 |      6.3230 |             0.8929 |      19.6613 | 
    6 | 00m01s |    0.71038 |          8.0228 |     12.7685 |             0.8963 |      25.0315 | 
    7 | 00m00s |    0.75883 |          3.7573 |      8.3800 |             0.1256 |       7.7688 | 
    8 | 00m00s |    0.75118 |          2.9976 |      3.5358 |             0.2072 |      15.9204

In [77]:
# params = lt.clean_param(lgbmBO.res['max']['max_params'])
# lgbm_model = LGBMClassifier(**params )
# lgbm_model.fit(x_pci_train, y_pci_train)
# y_pci_pred = lgbm_model.predict(x_pci_test)
# predictions = [round(value) for value in y_pci_pred]
# accuracy = accuracy_score(y_pci_test, predictions)
# print(1-accuracy)

0.24007285974499093


In [18]:
lgbm_params = {'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'min_child_samples': 20,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'num_leaves': 36,
 'min_child_weight': 0,
 'max_depth': 2,
 'learning_rate': 0.09783958802256404}

In [19]:
lgbm_params = {'learning_rate' : 0.099387, 'max_depth' : 14, 'min_child_weight':0, 'num_leaves':5}
lgbm_model = LGBMClassifier(**lgbm_params)
lgbm_model.fit(x_pci_train, y_pci_train)
y_pci_pred = lgbm_model.predict(x_pci_test)
predictions = [round(value) for value in y_pci_pred]
accuracy = accuracy_score(y_pci_test, predictions)
print(1-accuracy)

0.23970856102003646


In [20]:
lgbm_params = {'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'min_child_samples': 20,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'num_leaves': 36,
 'min_child_weight': 0,
 'max_depth': 2,
 'learning_rate': 0.09783958802256404}

# Experiment 

In [21]:
nrmse_matrix = np.empty((4, 3, 34))
nrmse_matrix[:] = np.nan

In [22]:
import warnings
warnings.filterwarnings('ignore')

def get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test) :
    if 'xgb' in model_name : 
        return xgboost_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    elif 'lgbm' in model_name : 
        return lgbm_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    elif 'knn' in model_name : 
        return knn_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    elif 'svm' in model_name : 
        return svm_target(curr_x_train, curr_y_train, curr_x_test, curr_y_test)
    
def get_params_range(model_name) :
    if 'xgb' in model_name : 
        return {'learning_rate': (1, 12),
                'booster' : (1, 3),
                'gamma' : (0, 5),
                'max_depth': (3, 10),
                'min_child_weight': (1, 1),
                'max_delta_weight': (1, 12),
                'rate_drop': (0, 1)}
    elif 'knn' in model_name : 
        return {'weight': (0, 1),
                'algorithm' : (0, 3),
                'leaf_size' : (5, 50),
                'p': (1, 2),}
    elif 'svm' in model_name : 
        return {'C': (0.01, 1), 'random_state' : (0, 5)}
    elif 'lgbm' in model_name :
        return {'min_child_weight': (0.01, 1),
              'learning_rate': (1, 10),
              'max_depth': (-1, 10),
              'num_leaves': (5, 20)}

def reset_model(model_name, params=None) :
    if 'xgb' in model_name :
        return XGBClassifier(**xgboost_params) if params is None else XGBClassifier(**params)
    elif 'knn' in model_name :
        return KNeighborsClassifier(**knn_params)
    elif 'svm' in model_name :
        return svm.LinearSVC(**svm_params) if params is None else svm.LinearSVC(**params)
    else :
        return LGBMClassifier(**lgbm_params) if params is None else LGBMClassifier(**params)

In [23]:
model_name_list = ['xgboost', 'lgbm', 'knn', 'svm']
model_idx = 1
model_name = model_name_list[model_idx]

# Baseline 

In [113]:
t = get_target(model_name, x_pci_train, y_pci_train, x_pci_test, y_pci_test)
xgbBO = BayesianOptimization(t.evaluate, 
                             get_params_range(model_name),
                             random_state = random, 
                             verbose=1)

xgbBO.maximize(init_points=20, n_iter=3)
params = t.clean_param(xgbBO.res['max']['max_params'])

model = reset_model(model_name, params=params)
# model = reset_model(model_name)

model.fit(x_pci_train, y_pci_train)

for s in demo_config[6] :
    y_pred = model.predict(x_pci_test_dict[(6, s)])
    predictions = [round(value) for value in y_pred]
    err = 1-accuracy_score(y_pci_test_dict[(p, s)], predictions)
    nrmse_matrix[model_idx, 0, s] = err
    
pickle.dump(model, open("db/%s_%s_baseline.pickle.dat" % ('PCI', model_name), "wb"))
for x in nrmse_matrix[model_idx, 0, 1:]:
    print('%.3f' % x)

Initialization
---------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   booster |     gamma |   learning_rate |   max_delta_weight |   max_depth |   min_child_weight |   rate_drop | 
    1 | 00m04s |    0.74681 |    1.5671 |    2.0326 |          7.0588 |             3.1128 |      5.6152 |             1.0000 |      0.2094 | 
    2 | 00m00s |    0.69945 |    2.3863 |    2.3447 |          8.7896 |             8.9824 |      7.6769 |             1.0000 |      0.7307 | 
    3 | 00m04s |    0.74317 |    1.8809 |    1.3462 |          4.2000 |             9.6343 |      5.3789 |             1.0000 |      0.6511 | 
    4 | 00m05s |    0.72896 |    1.3137 |    1.4590 |          6.6191 |            11.6931 |      7.0096 |             1.0000 |      0.4790 | 
    5 | 00m00s |    0.70747 |    2.0893 |    2.2884 |         10.8224 |            10.3607 |      5.2807 |             1.0000 | 

# Independent 

In [109]:
for s in demo_config[6] :
    t = get_target(model_name, x_pci_train_dict[(6, s)], y_pci_train_dict[(6, s)], 
                   x_pci_test_dict[(6, s)], y_pci_test_dict[(6, s)])
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=20, n_iter=3)
    params = t.clean_param(xgbBO.res['max']['max_params'])
    
    model = reset_model(model_name, params=params)
    
#     model = reset_model(model_name)
    model.fit(x_pci_train_dict[(6, s)], y_pci_train_dict[(6, s)])

    y_pred = model.predict(x_pci_test_dict[(6, s)])
    predictions = [round(value) for value in y_pred]
    err = 1-accuracy_score(y_pci_test_dict[(6, s)], predictions)
    nrmse_matrix[model_idx, 1, s] = err
    
    pickle.dump(model, open("db/%s_%s_independent_set_%s.pickle.dat" % ('PCI', model_name, s), "wb"))

for x in nrmse_matrix[model_idx, 1, 1:]:
    print('%.3f' % x)

0.170
0.327
0.271
0.215
0.346
0.389
0.407
0.378
0.314
0.303
0.182
0.195
0.296
0.329
0.360
0.333
0.228
0.178
0.381
0.256
0.305
0.238
0.337
0.293
0.301
0.321
0.412
0.227
0.269
0.228
0.277
0.208
0.329


# Transfer 

In [117]:
for s in demo_config[6] :
    curr_x_pci_train = pci_data[pci_data.set!=s].drop(['PCI', 'set'], axis=1)
    curr_y_pci_train = pci_data[pci_data.set!=s].PCI.apply(lambda x : pci_encode[x]).values.tolist()
    
#     Testing 100%
#     curr_x_pci_test = pci_data[pci_data.set==s].drop(['PCI', 'set'], axis=1)
#     curr_y_pci_test = pci_data[pci_data.set==s].PCI.apply(lambda x : pci_encode[x]).values.tolist()
#     Testing 30%
    curr_x_pci_test = x_pci_test_dict[(6, s)]
    curr_y_pci_test = y_pci_test_dict[(6, s)]
    curr_x_pci_test = curr_x_pci_test.drop(['set'], axis=1)

    t = get_target(model_name, curr_x_pci_train, curr_y_pci_train, curr_x_pci_test, curr_y_pci_test)
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=20, n_iter=3)
    params = t.clean_param(xgbBO.res['max']['max_params'])
    
    model = reset_model(model_name, params=params)
#     model = reset_model(model_name)

    model.fit(curr_x_pci_train, curr_y_pci_train)

    y_pred = model.predict(curr_x_pci_test)
    predictions = [round(value) for value in y_pred]
    err = 1-accuracy_score(curr_y_pci_test, predictions)
    nrmse_matrix[model_idx, 2, s] = err
    print('%.3f' % err)
    
    pickle.dump(model, open("db/%s_%s_transfer_except_%s.pickle.dat" % ('PCI', model_name, s), "wb"))
    
# for x in nrmse_matrix[model_idx, 2, 1:]:
#     print('%.3f' % x)

0.128
0.212
0.224
0.190
0.321
0.400
0.297
0.333
0.198
0.132
0.091
0.208
0.210
0.247
0.281
0.250
0.304
0.260
0.274
0.154
0.183
0.238
0.253
0.207
0.217
0.235
0.271
0.280
0.179
0.152
0.193
0.208
0.329


# Automatic Experiment 

In [24]:
random = 0
for model_idx in range(0, 4) :
    model_name = model_name_list[model_idx]
    print(model_name)
    
#     Baseline
    t = get_target(model_name, x_pci_train, y_pci_train, x_pci_test, y_pci_test)
    xgbBO = BayesianOptimization(t.evaluate, 
                                 get_params_range(model_name),
                                 random_state = random, 
                                 verbose=0)

    xgbBO.maximize(init_points=20, n_iter=3)
    params = t.clean_param(xgbBO.res['max']['max_params'])

    model = reset_model(model_name, params=params)
    # model = reset_model(model_name)

    model.fit(x_pci_train, y_pci_train)

    for s in demo_config[6] :
        y_pred = model.predict(x_pci_test_dict[(6, s)])
        predictions = [round(value) for value in y_pred]
        err = 1-accuracy_score(y_pci_test_dict[(p, s)], predictions)
        nrmse_matrix[model_idx, 0, s] = err

    pickle.dump(model, open("db/%s_%s_baseline.pickle.dat" % ('PCI', model_name), "wb"))
    
# Independent
    for s in demo_config[6] :
        t = get_target(model_name, x_pci_train_dict[(6, s)], y_pci_train_dict[(6, s)], 
                       x_pci_test_dict[(6, s)], y_pci_test_dict[(6, s)])
        xgbBO = BayesianOptimization(t.evaluate, 
                                     get_params_range(model_name),
                                     random_state = random, 
                                     verbose=0)

        xgbBO.maximize(init_points=20, n_iter=3)
        params = t.clean_param(xgbBO.res['max']['max_params'])

        model = reset_model(model_name, params=params)

    #     model = reset_model(model_name)
        model.fit(x_pci_train_dict[(6, s)], y_pci_train_dict[(6, s)])

        y_pred = model.predict(x_pci_test_dict[(6, s)])
        predictions = [round(value) for value in y_pred]
        err = 1-accuracy_score(y_pci_test_dict[(6, s)], predictions)
        nrmse_matrix[model_idx, 1, s] = err

        pickle.dump(model, open("db/%s_%s_independent_set_%s.pickle.dat" % ('PCI', model_name, s), "wb"))

# Transfer
    for s in demo_config[6] :
        curr_x_pci_train = pci_data[pci_data.set!=s].drop(['PCI', 'set'], axis=1)
        curr_y_pci_train = pci_data[pci_data.set!=s].PCI.apply(lambda x : pci_encode[x]).values.tolist()

    #     Testing 100%
    #     curr_x_pci_test = pci_data[pci_data.set==s].drop(['PCI', 'set'], axis=1)
    #     curr_y_pci_test = pci_data[pci_data.set==s].PCI.apply(lambda x : pci_encode[x]).values.tolist()
    #     Testing 30%
        curr_x_pci_test = x_pci_test_dict[(6, s)]
        curr_y_pci_test = y_pci_test_dict[(6, s)]
        curr_x_pci_test = curr_x_pci_test.drop(['set'], axis=1)

        t = get_target(model_name, curr_x_pci_train, curr_y_pci_train, curr_x_pci_test, curr_y_pci_test)
        xgbBO = BayesianOptimization(t.evaluate, 
                                     get_params_range(model_name),
                                     random_state = random, 
                                     verbose=0)

        xgbBO.maximize(init_points=20, n_iter=3)
        params = t.clean_param(xgbBO.res['max']['max_params'])

        model = reset_model(model_name, params=params)
    #     model = reset_model(model_name)

        model.fit(curr_x_pci_train, curr_y_pci_train)

        y_pred = model.predict(curr_x_pci_test)
        predictions = [round(value) for value in y_pred]
        err = 1-accuracy_score(curr_y_pci_test, predictions)
        nrmse_matrix[model_idx, 2, s] = err
#         print('%.3f' % err)

        pickle.dump(model, open("db/%s_%s_transfer_except_%s.pickle.dat" % ('PCI', model_name, s), "wb"))


xgboost
lgbm
knn
svm


# Model Analysis

In [25]:
scenarios = ['baseline', 'independent', 'transfer']
for scenario in range(3) :
    print("========================================"+scenarios[scenario])
    baseline_metrics = nrmse_matrix[:, scenario, 1:]
    baseline_metrics = baseline_metrics[~np.isnan(baseline_metrics)].reshape((4,len(sets)))
    print('avg of each methods', [ '%.3f' % elem for elem in np.mean(baseline_metrics, axis=1)])
    print('best performance', np.unique(np.argmin(baseline_metrics, axis=0), return_counts=True))
    
    print('diff performance')
    min_list = np.min(baseline_metrics, axis=0)

    for x in baseline_metrics :
        diff = x-min_list
        print('%.3f' % np.mean(diff[np.nonzero(diff)]))

========================================baseline
avg of each methods ['0.244', '0.236', '0.306', '0.267']
best performance (array([0, 1, 3]), array([16, 14,  3]))
diff performance
0.028
0.019
0.076
0.042
========================================independent
avg of each methods ['0.266', '0.279', '0.295', '0.308']
best performance (array([0, 1, 2, 3]), array([16,  9,  2,  6]))
diff performance
0.028
0.042
0.051
0.078
========================================transfer
avg of each methods ['0.238', '0.231', '0.264', '0.237']
best performance (array([0, 1, 3]), array([14,  9, 10]))
diff performance
0.040
0.038
0.052
0.036


# Scenario Analysis 

In [26]:
for curr_model in range(4) :
    print("========================================"+model_name_list[curr_model])
    metrics = nrmse_matrix[curr_model, :, 1:]
    metrics = metrics[~np.isnan(metrics)].reshape((3,len(sets)))
    print('avg of each scenario', [ '%.3f' % elem for elem in np.mean(metrics, axis=1)])
    print('best performance', np.unique(np.argmin(metrics, axis=0), return_counts=True))
    
    transfer_metrics = nrmse_matrix[curr_model, 2, 1:]
    for i in range(2) :
        curr_metrics = nrmse_matrix[curr_model, i, 1:]
        print('[%s]diff avg %.3f' % (scenarios[i], 
                                        np.mean(curr_metrics - transfer_metrics)))
        
        improve_idx = transfer_metrics < curr_metrics
        diff = transfer_metrics[improve_idx] - curr_metrics[improve_idx]
        print('[%s]+++++++increase %d %.3f' % (scenarios[i], len(diff), np.mean(diff)))
        
        drop_idx = transfer_metrics > curr_metrics
        diff = transfer_metrics[drop_idx] - curr_metrics[drop_idx]
        print('[%s]-------decrease %d %.3f' % (scenarios[i], len(diff), np.mean(diff)))

========================================xgboost
avg of each scenario ['0.244', '0.266', '0.238']
best performance (array([0, 1, 2]), array([ 9,  8, 16]))
[baseline]diff avg 0.006
[baseline]+++++++increase 19 -0.027
[baseline]-------decrease 10 0.034
[independent]diff avg 0.028
[independent]+++++++increase 24 -0.057
[independent]-------decrease 8 0.055
========================================lgbm
avg of each scenario ['0.236', '0.279', '0.231']
best performance (array([0, 1, 2]), array([12,  2, 19]))
[baseline]diff avg 0.005
[baseline]+++++++increase 20 -0.028
[baseline]-------decrease 10 0.040
[independent]diff avg 0.048
[independent]+++++++increase 24 -0.077
[independent]-------decrease 6 0.046
========================================knn
avg of each scenario ['0.306', '0.295', '0.264']
best performance (array([0, 1, 2]), array([ 3,  9, 21]))
[baseline]diff avg 0.042
[baseline]+++++++increase 29 -0.049
[baseline]-------decrease 2 0.019
[independent]diff avg 0.031
[independent]+++++++in

In [29]:
save_to_pickle(nrmse_matrix, 'nrmse_matrix_pci')

In [30]:
test = load_from_pickle('nrmse_matrix_pci')
for x in test[1, 0, 1:]:
    print('%.3f' % x)

0.149
0.239
0.224
0.215
0.346
0.322
0.275
0.333
0.233
0.145
0.104
0.169
0.222
0.271
0.326
0.298
0.177
0.205
0.274
0.205
0.232
0.238
0.277
0.232
0.205
0.259
0.282
0.227
0.218
0.177
0.193
0.195
0.318


In [28]:
for x in nrmse_matrix[1, 0, 1:]:
    print('%.3f' % x)

0.149
0.239
0.224
0.215
0.346
0.322
0.275
0.333
0.233
0.145
0.104
0.169
0.222
0.271
0.326
0.298
0.177
0.205
0.274
0.205
0.232
0.238
0.277
0.232
0.205
0.259
0.282
0.227
0.218
0.177
0.193
0.195
0.318


# Generate Predicted Coordinate 

In [70]:
x_cut = 50  
y_cut = 100 

old_origin_img = cv2.imread('../image/map.png',0)
crop = old_origin_img[y_cut:318, x_cut:927]
crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2RGB)

x_coord_list = []
y_coord_list = []
pci_list = []
for lon in range(0, crop.shape[1]) :
    for lat in range(0, crop.shape[0]) :
        x_coord_list.append(x_cut + lon)
        y_coord_list.append(y_cut + lat)
        
all_x_pci = pd.DataFrame({'location_x':x_coord_list, 'location_y':y_coord_list})

# Bayesian Opt 

In [102]:
from matplotlib import gridspec

loc_df = pci_data[['location_x', 'location_y', 'PCI']]
def get_pci(x, y) :
    distance = lambda d: math.hypot(abs(x-d[0]), abs(y-d[1]))
    loc_df["d"] = loc_df.apply(distance, axis=1)
    return loc_df.loc[loc_df.d.idxmin()].PCI

class target() :
    def optimize(self, x, y) :
        if self.bayes_opt is None or self.bayes_opt.X is None:
            return 1000

        bo = self.bayes_opt
        bo.gp.fit(bo.X, bo.Y)
        mu, sigma = bo.gp.predict(all_x_pci.values, return_std=True)
        return -np.mean(sigma)

def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    plot(sigma)
    plt.show()
    return mu, sigma

def plot_gp(bo, x, curr_x_train, curr_y_train, set_val, model, show_sigma_map=False):
    path = "../results/predicted/pci/real_%s_set_%d.png" % (model, set_val)
    background = get_map_image()
    p_color = [pci_decode[y] for y in curr_y_train]
    p_color = [pci_color_dict[y] if y in pci_color_dict else (255, 255, 255) for y in p_color]
    b = visualize(background, curr_x_train['location_x'].astype(int), curr_x_train['location_y'].astype(int), 
                  p_color, path, adjustment=True)

    if show_sigma_map :
        normalize_sigma = matplotlib.colors.Normalize(vmin=min(sigma), vmax=max(sigma))
        mu_map = [cmap(normalize_sigma(value))[:3] for value in mu_sigma]
        mu_map = [[int(x*255) for x in value] for value in mu_map]    
        a=visualize_all_location_heatmap(a, x_coord_view, y_coord_view, mu_map, 
                                         cmap, normalize_sigma, filename=None,
                                         size=1, figsize=(20,10), adjustment=False, show=False)

In [25]:
random = 0
t = target()
bo2 = BayesianOptimization(t.optimize, {'x': (min(x_coord_list), max(x_coord_list)), 
                                        'y': (min(y_coord_list), max(y_coord_list))},
                           random_state=random, 
                           verbose=1)
t.bayes_opt = bo2

iterations = 100
gp_params = {"alpha": 1e-5, "n_restarts_optimizer": 3, 'random_state':random}
bo2.maximize(init_points=2, n_iter=iterations, acq="ei", xi=0.1, **gp_params)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    1 | 00m00s | 1000.00000 |  530.7606 |  230.7997 | 
    2 | 00m00s |   -0.99999 |  676.5059 |  218.2397 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    3 | 00m00s |   -0.99998 |  689.4791 |  189.5332 | 
    4 | 00m00s |   -0.98814 |  530.4559 |  231.3546 | 
    5 | 00m00s |   -1.00000 |   72.7881 |  228.1264 | 
    6 | 00m00s |   -1.00000 |  617.4562 |  261.6594 | 
    7 | 00m00s |   -1.00000 |  837.4735 |  285.0548 | 
    8 | 00m00s |   -1.00000 |   61.4990 |  274.8175 | 
    9 | 00m00s |   -1.00000 |  702.4929 |  159.9914 | 
   10 | 00m00s |   -1.00000 |  463.5776 |  267.2542 | 
   11 | 00m00s |   -1.00000 |  645.7080 |  121.2919 | 
   12 | 00m00s |   -1.00000 |  270.6671 |  108.0514 | 
   13 | 00m00s |   -1.00000 |  110.9284 |  167.7725 | 
   14 | 00m00s |   -1.00000 | 

In [73]:
random = 3
t = target()
bo3 = BayesianOptimization(t.optimize, {'x': (min(x_coord_list), max(x_coord_list)), 
                                        'y': (min(y_coord_list), max(y_coord_list))},
                           random_state=random, 
                           verbose=1)
t.bayes_opt = bo3

iterations = 400
gp_params = {"alpha": 1e-5, "n_restarts_optimizer": 3, 'random_state':random}
bo3.maximize(init_points=10, n_iter=iterations, acq="ei", xi=1e+1, **gp_params)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    1 | 00m00s | 1000.00000 |  532.4990 |  106.4831 | 
    2 | 00m00s |   -0.99999 |  670.3375 |  199.1328 | 
    3 | 00m00s |   -0.99998 |  304.8326 |  240.8643 | 
    4 | 00m00s |   -0.99998 |  497.4850 |  160.4317 | 
    5 | 00m00s |   -0.99997 |  832.2215 |  246.7473 | 
    6 | 00m00s |   -0.99996 |  835.1527 |  228.2172 | 
    7 | 00m00s |   -0.99968 |  160.0127 |  105.2041 | 
    8 | 00m00s |   -0.99965 |  231.5448 |  221.2713 | 
    9 | 00m00s |   -0.99968 |   95.0853 |  156.2578 | 
   10 | 00m00s |   -0.99346 |  436.1494 |  190.0770 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
   11 | 00m00s |   -0.99915 |  191.6835 |  118.1789 | 
   12 | 00m00s |   -0.99907 |  685.8851 |  220.6383 | 
   13 | 00m00s |   -0.99898 |  759.7297 |  189.2084 | 
   14 | 00m00s |   -0.99890 | 

  145 | 00m02s |   -0.97503 |  376.5080 |  184.9413 | 
  146 | 00m02s |   -0.97472 |  704.1287 |  132.2572 | 
  147 | 00m02s |   -0.97453 |  819.5426 |  186.7378 | 
  148 | 00m02s |   -0.97424 |  459.8544 |  261.8791 | 
  149 | 00m02s |   -0.97403 |  698.4592 |  195.9000 | 
  150 | 00m02s |   -0.97380 |  211.1492 |  139.4483 | 
  151 | 00m02s |   -0.97350 |  539.5835 |  153.9385 | 
  152 | 00m02s |   -0.97331 |  355.3220 |  310.5848 | 
  153 | 00m02s |   -0.97304 |  634.3614 |  149.0565 | 
  154 | 00m02s |   -0.97278 |  913.7160 |  246.9051 | 
  155 | 00m02s |   -0.97260 |  715.7100 |  187.0292 | 
  156 | 00m02s |   -0.97241 |  921.6908 |  236.8554 | 
  157 | 00m02s |   -0.97218 |  751.2969 |  214.1380 | 
  158 | 00m02s |   -0.97196 |  619.0876 |  118.9626 | 
  159 | 00m02s |   -0.97177 |  368.4802 |  194.4183 | 
  160 | 00m02s |   -0.97151 |  816.7489 |  213.2403 | 
  161 | 00m02s |   -0.97131 |  764.0849 |  259.5815 | 
  162 | 00m02s |   -0.97109 |  285.6034 |  149.6087 | 
  163 | 00

  294 | 00m06s |   -0.97100 |  906.1399 |  176.2233 | 
  295 | 00m06s |   -0.97090 |  648.9461 |  287.5102 | 
  296 | 00m06s |   -0.97078 |  760.2599 |  249.9943 | 
  297 | 00m06s |   -0.97065 |  876.3932 |  115.1651 | 
  298 | 00m06s |   -0.97054 |  666.4305 |  236.0795 | 
  299 | 00m06s |   -0.97042 |   59.1592 |  205.0674 | 
  300 | 00m06s |   -0.97031 |  757.7329 |  285.7268 | 
  301 | 00m06s |   -0.97015 |  894.6530 |  171.9127 | 
  302 | 00m06s |   -0.97002 |  199.6354 |  142.1749 | 
  303 | 00m06s |   -0.96990 |  172.4190 |  283.0682 | 
  304 | 00m06s |   -0.96979 |  380.6040 |  133.4013 | 
  305 | 00m06s |   -0.96969 |  823.2894 |  169.0986 | 
  306 | 00m07s |   -0.96959 |  278.5622 |  169.2848 | 
  307 | 00m07s |   -0.96942 |  696.0400 |  142.2999 | 
  308 | 00m08s |   -0.96931 |  819.5601 |  105.3234 | 
  309 | 00m06s |   -0.96917 |  729.6829 |  248.3993 | 
  310 | 00m07s |   -0.96905 |  807.0583 |  103.1906 | 
  311 | 00m07s |   -0.96890 |  787.5849 |  101.1791 | 
  312 | 00

# Bayesian Independent 

In [90]:
acc_dict = {}
for set_val in demo_config[6] :
    curr_pci_data = pci_data[pci_data.set == set_val]
    iterations = int(0.2*len(curr_pci_data)) + 5

    temp = curr_pci_data.copy()
    temp2 = pd.DataFrame(columns=temp.columns)
    for x in bo3.X[:iterations] :
        distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
        temp["d"] = temp.apply(distance, axis=1)
        temp2 = temp2.append(temp.loc[temp.d.idxmin()])

    temp3 = curr_pci_data[~curr_pci_data.index.isin(temp2.index)]

    curr_x_train = temp2.drop(["PCI", "d"], axis=1)
    curr_y_train = temp2.PCI.apply(lambda x : pci_encode[x]).values.tolist()
    curr_x_test = temp3.drop("PCI", axis=1)
    curr_y_test = temp3.PCI.apply(lambda x : pci_encode[x]).values.tolist()

#     plot_gp(bo2, all_x_pci.values, curr_x_train, curr_y_train, set_val, "xgboost")
    
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':4.2522, 
#               'max_delta_weight':11, 'random_state' :random}
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':1, 
#               'max_delta_weight':11, 'random_state' :random}

#     t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
#     xgbBO = BayesianOptimization(t.evaluate, 
#                                  get_params_range(model_name),
#                                  random_state = random, 
#                                  verbose=0)

#     xgbBO.maximize(init_points=5, n_iter=3)
#     print(xgbBO.res['max']['max_params'])
#     params = t.clean_param(xgbBO.res['max']['max_params'])

    params = lgbm_params
    params['min_data_in_bin']=1
    params['min_data']=1
    
    model = reset_model(model_name, params)
    model.fit(curr_x_train, curr_y_train)
    pickle.dump(model, open("db/%s_%s_bayesian_independent_%s.pickle.dat" % \
                            ('PCI', model_name, set_val), "wb"))

# for set_val in demo_config[6] :
    y_pci_pred = model.predict(curr_x_test)
    predictions = [round(value) for value in y_pci_pred]
    accuracy = accuracy_score(curr_y_test, predictions)
    acc_dict[set_val] = [len(curr_x_train), len(curr_x_test), accuracy]
    print(1-accuracy)

0.2355212355212355
0.32907348242811496
0.3122362869198312
0.31818181818181823
0.3244444444444444
0.4056224899598394
0.4274509803921569
0.4007936507936508
0.37815126050420167
0.23696682464454977
0.2710280373831776
0.2155963302752294
0.3083700440528634
0.38912133891213385
0.3780487804878049
0.3819742489270386
0.2995391705069125
0.25
0.28755364806866957
0.27188940092165903
0.35398230088495575
0.32432432432432434
0.3318965517241379
0.35398230088495575
0.30603448275862066
0.32173913043478264
0.3620689655172413
0.2932692307692307
0.2844036697247706
0.28959276018099545
0.31759656652360513
0.2990654205607477
0.30932203389830504


In [29]:
# lgbm, random state 0 
bayes_inden = np.array([x for x in acc_dict.values()])
for x in list(bayes_inden[:, 2]) :
    print(1-x)

0.2934362934362934
0.3482428115015974
0.35443037974683544
0.35
0.4311111111111111
0.48594377510040165
0.4627450980392157
0.503968253968254
0.37914691943127965
0.4299065420560748
0.5154185022026432
0.4518828451882845
0.4796747967479674
0.502145922746781
0.3548387096774194
0.3495575221238938
0.36036036036036034
0.44396551724137934
0.4070796460176991
0.43534482758620685
0.5
0.5043103448275862
0.3557692307692307
0.4862385321100917
0.3846153846153846
0.37768240343347637
0.3271028037383178
0.4491525423728814


# Bayesian Baseline 

In [30]:
acc_dict = {}
all_curr_x_train, all_curr_y_train = pd.DataFrame(), []
all_curr_x_test, all_curr_y_test = pd.DataFrame(), []
all_curr_x_test_dict, all_curr_y_test_dict = {}, {}
for set_val in demo_config[6] :
    curr_pci_data = pci_data[pci_data.set == set_val]
    iterations = int(0.2*len(curr_pci_data))

    temp = curr_pci_data.copy()
    temp2 = pd.DataFrame(columns=temp.columns)
    for x in bo3.X[:iterations] :
        distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
        temp["d"] = temp.apply(distance, axis=1)
        temp2 = temp2.append(temp.loc[temp.d.idxmin()])

    temp3 = curr_pci_data[~curr_pci_data.index.isin(temp2.index)]

    curr_x_train = temp2.drop(["PCI", "d"], axis=1)
    curr_y_train = temp2.PCI.apply(lambda x : pci_encode[x]).values.tolist()
    curr_x_test = temp3.drop("PCI", axis=1)
    curr_y_test = temp3.PCI.apply(lambda x : pci_encode[x]).values.tolist()

    all_curr_x_train = all_curr_x_train.append(curr_x_train)
    all_curr_y_train += curr_y_train 
    all_curr_x_test = all_curr_x_test.append(curr_x_test)
    all_curr_y_test += curr_y_test
    all_curr_x_test_dict[set_val] = curr_x_test
    all_curr_y_test_dict[set_val] = curr_y_test  

#     plot_gp(bo2, all_x_pci.values, curr_x_train, curr_y_train, set_val, "xgboost")
    
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':4.2522, 
#               'max_delta_weight':11, 'random_state' :random}
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':1, 
#               'max_delta_weight':11, 'random_state' :random}

t = get_target(model_name, curr_x_train, curr_y_train, curr_x_test, curr_y_test)
xgbBO = BayesianOptimization(t.evaluate, 
                             get_params_range(model_name),
                             random_state = random, 
                             verbose=1)

xgbBO.maximize(init_points=5, n_iter=15)
print(xgbBO.res['max']['max_params'])
params = t.clean_param(xgbBO.res['max']['max_params'])

# params = lgbm_params
params['min_data_in_bin']=1
params['min_data']=1
    
model = reset_model(model_name, params)
model.fit(curr_x_train, curr_y_train)
pickle.dump(model, open("db/%s_%s_bayesian_baseline_%s.pickle.dat" % ('PCI', model_name, set_val), "wb"))

for set_val in demo_config[6] :
    y_pci_pred = model.predict(all_curr_x_test_dict[set_val])
    predictions = [round(value) for value in y_pci_pred]
    accuracy = accuracy_score(all_curr_y_test_dict[set_val], predictions)
    acc_dict[set_val] = [len(curr_x_train), len(curr_x_test), accuracy]
    print(1-accuracy)

Initialization
---------------------------------------------------------------------------------
 Step |   Time |      Value |   algorithm |   leaf_size |         p |    weight | 
    1 | 00m00s |    0.58506 |      2.6889 |      6.3444 |    1.5909 |    0.5508 | 
    2 | 00m00s |    0.58506 |      0.3768 |     25.5575 |    1.0240 |    0.7081 | 
    3 | 00m00s |    0.58506 |      0.6217 |     34.2115 |    1.5589 |    0.2909 | 
    4 | 00m00s |    0.58506 |      0.1544 |     17.5319 |    1.2593 |    0.5108 | 
    5 | 00m00s |    0.58506 |      1.3224 |     35.4315 |    1.4151 |    0.8929 | 
Bayesian Optimization
---------------------------------------------------------------------------------
 Step |   Time |      Value |   algorithm |   leaf_size |         p |    weight | 
    6 | 00m08s |    0.58506 |      2.7451 |     31.7164 |    1.5186 |    0.6827 | 
    7 | 00m06s |    0.58506 |      0.7536 |     29.4385 |    1.9309 |    0.6573 | 
    8 | 00m00s |    0.58506 |      0.4244 |     26.4

In [31]:
# lgbm, random state 0 
bayes_baseline = np.array([x for x in acc_dict.values()])
for x in list(bayes_baseline[:, 2]) :
    print(1-x)

0.3916349809885932
0.4668769716088328
0.42148760330578516
0.3973214285714286
0.48245614035087714
0.5375494071146245
0.49224806201550386
0.48046875
0.4672897196261683
0.45871559633027525
0.4869565217391304
0.4403292181069959
0.4661354581673307
0.5
0.44545454545454544
0.40174672489082974
0.3733333333333333
0.41276595744680855
0.4217391304347826
0.47457627118644063
0.4678111587982833
0.5021097046413502
0.44131455399061037
0.509009009009009
0.45333333333333337
0.4514767932489452
0.46543778801843316
0.41493775933609955


# Bayesian Transfer 

In [32]:
acc_dict = {}
all_curr_x_train_dict, all_curr_y_train_dict = {}, {}
all_curr_x_test_dict, all_curr_y_test_dict = {}, {}
for set_val in demo_config[6] :
    curr_pci_data = pci_data[pci_data.set == set_val]
    iterations = int(0.7*len(curr_pci_data)) + 5

    temp = curr_pci_data.copy()
    temp2 = pd.DataFrame(columns=temp.columns)
    for x in bo3.X[:iterations] :
        distance = lambda d: math.hypot(abs(x[0]-d[0]), abs(x[1]-d[1]))
        temp["d"] = temp.apply(distance, axis=1)
        temp2 = temp2.append(temp.loc[temp.d.idxmin()])

    curr_x_train = temp2.drop(["PCI", "d"], axis=1)
    curr_y_train = temp2.PCI.apply(lambda x : pci_encode[x]).values.tolist()
 
    all_curr_x_train_dict[set_val] = curr_x_train
    all_curr_y_train_dict[set_val] = curr_y_train  
    all_curr_x_test_dict[set_val] = curr_pci_data.drop("PCI", axis=1)
    all_curr_y_test_dict[set_val] = curr_pci_data.PCI.apply(lambda x : pci_encode[x]).values.tolist()
#     print(set_val, len(curr_x_train), len(curr_x_test))

#     plot_gp(bo2, all_x_pci.values, curr_x_train, curr_y_train, set_val, "xgboost")
    
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':4.2522, 
#               'max_delta_weight':11, 'random_state' :random}
#     params = {'learning_rate' : 0.03, 'max_depth' : 9, 'min_child_weight':1, 'gamma':1, 
#               'max_delta_weight':11, 'random_state' :random}

#     t = get_target(model_name, all_curr_x_train, all_curr_y_train, all_curr_x_test, all_curr_y_test)
#     xgbBO = BayesianOptimization(t.evaluate, 
#                                  get_params_range(model_name),
#                                  random_state = random, 
#                                  verbose=0)

#     xgbBO.maximize(init_points=5, n_iter=3)
#     print(xgbBO.res['max']['max_params'])
#     params = t.clean_param(xgbBO.res['max']['max_params'])

    params = lgbm_params
    params['min_data_in_bin']=1
    params['min_data']=1
    
for set_val in demo_config[6] :
    curr_x_train, curr_y_train = pd.DataFrame(), []
    for k in all_curr_x_train_dict :
        if k != set_val :
            curr_x_train = curr_x_train.append(all_curr_x_train_dict[k])
            curr_y_train += all_curr_y_train_dict[k]
    
    model = reset_model(model_name, params)
    model.fit(curr_x_train, curr_y_train)
    pickle.dump(model, open("db/%s_%s_bayesian_transfer_%s.pickle.dat" % ('PCI', model_name, set_val), "wb"))

    y_pci_pred = model.predict(all_curr_x_test_dict[set_val])
    predictions = [round(value) for value in y_pci_pred]
    accuracy = accuracy_score(all_curr_y_test_dict[set_val], predictions)
    acc_dict[set_val] = [len(curr_x_train), len(all_curr_y_test_dict[set_val]), accuracy]
    print(1-accuracy)

0.207667731629393
0.2853333333333333
0.27561837455830385
0.24904214559386972
0.3097014925373134
0.37583892617449666
0.32450331125827814
0.3076923076923077
0.19920318725099606
0.2165354330708661
0.25555555555555554
0.2978723404255319
0.3129251700680272
0.3107142857142857
0.2645914396887159
0.2509225092250923
0.21590909090909094
0.25724637681159424
0.2878228782287823
0.30181818181818176
0.28624535315985133
0.3096085409252669
0.2570281124497992
0.2702702702702703
0.2395437262357415
0.2681159420289855
0.2549019607843137
0.28113879003558717


In [33]:
# lgbm, random state 0 
bayes_transfer = np.array([x for x in acc_dict.values()])
for x in list(bayes_transfer[:, 2]) :
    print(1-x)

0.207667731629393
0.2853333333333333
0.27561837455830385
0.24904214559386972
0.3097014925373134
0.37583892617449666
0.32450331125827814
0.3076923076923077
0.19920318725099606
0.2165354330708661
0.25555555555555554
0.2978723404255319
0.3129251700680272
0.3107142857142857
0.2645914396887159
0.2509225092250923
0.21590909090909094
0.25724637681159424
0.2878228782287823
0.30181818181818176
0.28624535315985133
0.3096085409252669
0.2570281124497992
0.2702702702702703
0.2395437262357415
0.2681159420289855
0.2549019607843137
0.28113879003558717


In [33]:
import bayes_opt